In [ ]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Parcial 2016 2do Cuatrimestre, Ejercicio 2 

En este ejercicio queremos programar un sistema que recomiende
textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto
es un string de longitud variable. 

Además contamos con un RDD que
indica qué términos le gustan o no a cada usuario de la forma (userId,
término, score) por ejemplo (23, “calesita”, -2). 

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para
los términos que aparecen en el documento. 

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en
estos casos simplemente los consideramos neutros (score=0)

In [ ]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [ ]:
documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [ ]:
# generamos algo del tipo (word, docId) donde word sera nuestra key
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()]).collect()

In [ ]:
# llevamos scores a una representacion que nos permita unir la informacion
scoresRDDForJoin = scoresRDD.map(lambda a: (a[1],(a[0],a[2])))
scoresRDDForJoin.collect()

In [ ]:
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .collect()

In [ ]:
# para poder realizar una acumulacion debemos llevar la informacion a la representacion (docId, userId, score) 
# y luego realizar una acumulacion de scores
# ignorando el score

#notese que el K es docId, userId y el V es el score que aporta el termino

documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .collect()

In [ ]:
# acumulamos haciendo reduce
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .reduceByKey(lambda a,b: a + b)\
    .collect()

In [ ]:
# llevamos a la representacion pedida
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .reduceByKey(lambda a,b: a + b)\
    .map(lambda a: (a[0][0], a[0][1], a[1]))\
    .collect()